In [24]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime
from scipy.optimize import minimize

## Group Assignment
### Team Number: 12
### Team Member Names: Sharuga, Derek, Alex
### Team Strategy Chosen: Market Meet

Disclose any use of AI for this assignment below (detail where and how you used it).  Please see the course outline for acceptable uses of AI.


In [25]:
START_DATE = '2023-11-25'
END_DATE = '2024-11-23'
INVESTMENT = 1000000


In [26]:
# Function to calculate the purchasing fee depending on the number of shares
def calculate_purchase_fee(shares):
    return max(3.95, 0.01*shares)

In [27]:

def validity(tickers):
    final_list = []
    for ticker in tickers:
        #only append the stock to the final stock list if it is valid
        if (not check_delist(ticker) and
            check_currency(ticker) and
            check_volume(ticker)):
                final_list.append(ticker)
    return final_list

def check_delist(ticker):
    stock = yf.Ticker(ticker)
    try:
        data = stock.history(period='1d')
        if data.empty:
            #if we can't find any data on the stock, it's delisted
            return True
        else:
            #check that there is actually valid market data for this stock - AI
            if 'Close' not in data.columns or data['Close'].isnull().all():
                #if there are no valid close prices, the stock is delisted
                return True
            return False
    except Exception as e:
        #if there is an error in finding the stock's data, we can assume that it's delisted
        return True

def check_volume(ticker):
    volume = yf.Ticker(ticker).history(period='1d')['Volume']
    avg_monthly_volume = volume.resample('ME').mean()
    return avg_monthly_volume.mean() >= 100000

def check_currency(ticker):
    """Check if the stock is in USD or CAD currency."""
    stock = yf.Ticker(ticker)
    try:
        currency = stock.info.get('currency', None)
        return currency in ['USD', 'CAD']
    except Exception as e:
        return False  # If error, assume it’s invalid

all_data = pd.read_csv('Tickers_Example.csv', header=None)

all_data = pd.DataFrame(validity(all_data[0]))
all_data.rename(columns = {0:'Ticker'}, inplace=True)

$AGN: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$CELG: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$MON: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$RTN: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")


In [28]:
# Create a list of all the stock betas (retrieved from yfinance) of stock tickers in a list
def get_betas(tickers):
    betas = []
    
    for ticker in tickers:
        betas.append(yf.Ticker(ticker).info.get('beta'))

    return betas

In [29]:
def get_sector(tickers):
    sectors = []

    for ticker in tickers:
        sectors.append(yf.Ticker(ticker).info.get('sector'))

    return sectors

In [30]:
# Add a column containing all the stock betas to [all_data]
all_data = all_data.assign(Beta=get_betas(all_data['Ticker']))
all_data = all_data.assign(Sector=get_sector(all_data['Ticker']))

all_data

,Ticker,Beta,Sector
0,AAPL,1.240,Technology
1,ABBV,0.613,Healthcare
2,ABT,0.722,Healthcare
3,ACN,1.245,Technology
4,AIG,1.069,Financial Services
5,AMZN,1.146,Consumer Cyclical
6,AXP,1.214,Financial Services
7,BA,1.572,Industrials
8,BAC,1.325,Financial Services
9,BB.TO,1.068,Technology


In [31]:
# Choose the max(24, data.length) stocks that match the market the closest
def filter_betas(data):
    # Remove any negative betas, as long as data ends with at least 12 characters
    data.sort_values(by='Beta', axis=0, inplace=True, kind='quicksort')
    data.drop(index=data[(data['Beta'] <= 0) & (len(data) > 12)].index, inplace=True)
    data.reset_index(drop=True, inplace=True)

    data['temp'] = abs(1-data['Beta'])
    data.sort_values(by='temp', axis=0, inplace=True, kind='quicksort')    # Sort data by temp
    data.drop(columns='temp', axis=1, inplace=True)    # Remove temp

    data.index = range(0, len(data))    # Reassign the index
    
    return data[:24] if len(data) > 24 else data    # Return first 24 elements of data if it is greater than 24, other wise return data

In [32]:
stocks = [x for _, x in all_data.groupby('Sector')]
stocks = list(map(filter_betas, stocks))
stocks

[  Ticker   Beta                  Sector
 0   T.TO  0.722  Communication Services,
   Ticker   Beta             Sector
 0   AMZN  1.146  Consumer Cyclical,
   Ticker   Beta              Sector
 0     MO  0.670  Consumer Defensive
 1     KO  0.620  Consumer Defensive
 2     PM  0.560  Consumer Defensive
 3    PEP  0.542  Consumer Defensive
 4     CL  0.415  Consumer Defensive
 5     PG  0.414  Consumer Defensive,
   Ticker   Beta              Sector
 0    USB  1.040  Financial Services
 1     BK  1.060  Financial Services
 2    AIG  1.069  Financial Services
 3  RY.TO  0.842  Financial Services
 4  TD.TO  0.822  Financial Services
 5    AXP  1.214  Financial Services
 6    BLK  1.311  Financial Services
 7    BAC  1.325  Financial Services
 8      C  1.426  Financial Services
 9   PYPL  1.436  Financial Services,
   Ticker   Beta      Sector
 0    ABT  0.722  Healthcare
 1    PFE  0.615  Healthcare
 2   ABBV  0.613  Healthcare
 3    UNH  0.591  Healthcare
 4    BMY  0.441  Healthcare
 5

In [33]:
all_data = filter_betas(all_data)
all_data.index = all_data.index + 1

In [40]:
tsx60_info = yf.Ticker('XIU.TO')
tsx60 = tsx60_info.history(start=START_DATE, end=END_DATE, interval='1d')

sp500_info = yf.Ticker('^GSPC')
sp500 = sp500_info.history(start=START_DATE, end=END_DATE, interval='1d')
sp500 = sp500.reindex(tsx60.index, method='nearest')

market_returns = (tsx60['Close'].pct_change() + sp500['Close'].pct_change())/2
market_returns.index = pd.to_datetime(market_returns.index.strftime('%Y-%m-%d'))
market_returns = market_returns.dropna()
market_returns_mean = market_returns.mean()

market_returns_df = pd.DataFrame(market_returns)

display(market_returns_df.index)

DatetimeIndex(['2023-11-28', '2023-11-29', '2023-11-30', '2023-12-01',
               '2023-12-04', '2023-12-05', '2023-12-06', '2023-12-07',
               '2023-12-08', '2023-12-11',
               ...
               '2024-11-07', '2024-11-08', '2024-11-11', '2024-11-12',
               '2024-11-13', '2024-11-14', '2024-11-15', '2024-11-18',
               '2024-11-19', '2024-11-20'],
              dtype='datetime64[ns]', name='Date', length=247, freq=None)

In [137]:
def drop_months(df):
    # months = pd.to_datetime(df.index.to_numpy().astype('datetime64[M]'))
    months = df.resample('MS').bfill().index
    
    for month in months:
        temp = df[(df.index.month == month.month) & (df.index.year == month.year)].index
        if len(temp) < 18:
            df.drop(index=temp, inplace=True)

    return df

In [138]:
drop_months(market_returns)

Date
2023-12-01    0.007812
2023-12-04   -0.003026
2023-12-05    0.000038
2023-12-06   -0.004850
2023-12-07    0.003822
                ...   
2024-10-25   -0.002030
2024-10-28    0.004022
2024-10-29    0.000137
2024-10-30   -0.002456
2024-10-31   -0.016166
Name: Close, Length: 230, dtype: float64

In [35]:
# all_data
ticker_returns = pd.DataFrame(columns=all_data['Ticker'])
for ticker in all_data['Ticker']:
        data = yf.Ticker(ticker).history(start=START_DATE, end=END_DATE)['Close']
        ticker_returns[ticker] = data.pct_change().dropna()  # Calculate daily returns

ticker_returns.index = ticker_returns.index.strftime('%Y-%m-%d')

common_dates = ticker_returns.index.intersection(market_returns_df.index)
ticker_returns = ticker_returns.loc[common_dates]
market_returns_df = market_returns_df.loc[common_dates]

ticker_returns = ticker_returns.dropna()
market_returns_df = market_returns_df.dropna()

In [36]:
portfolio_data = []

#optimize for arbitray mean
#take into fees

def portfolio_generator(tickers):
    global portfolio_data
    min_result = float('inf')

    for i in range(12, 25):  
        selected_tickers = tickers[:i]  
        target_date = '2024-11-01'

        # Dictionary to store the closing prices
        ticker_prices = {}

        # Loop through each ticker and fetch its historical data
        for ticker in tickers:
            stock = yf.Ticker(ticker)
            history = stock.history(start=target_date, end=target_date)
            
            # Check if the target date exists in the historical data
            if not history.empty and target_date in history.index.strftime('%Y-%m-%d'):
                # Access the closing price for that date
                close_price = history.loc[history.index.strftime('%Y-%m-%d') == target_date, 'Close'].values[0]
                ticker_prices[ticker] = close_price
            else:
                ticker_prices[ticker] = None  # No data available for that date



        def objective(weights):
            return np.std(np.dot(ticker_returns[selected_tickers].dropna().values, weights) - market_returns_df.dropna().values)
        
        # Initial weights (equal distribution) - AI
        initial_weights = [1 / len(selected_tickers)] * len(selected_tickers)
        
        def sum_weights(weights):
            return sum(weights) -1
        
        def weight_bounds(weights):
            min_weight = 1 / (2 * len(weights))  # Minimum weight per stock
            max_weight = 0.15  # Maximum weight per stock
            return [
            min_weight - w if w < min_weight else w - max_weight if w > max_weight else 0 for w in weights]

        # Constraints: Weights must sum to 1 - AI
        constraints = [
            {'type': 'eq', 'fun': sum_weights},  # Sum of weights = 1
            {'type': 'eq', 'fun': weight_bounds}]

        
        # Bounds: Weights between 0 and 1 - AI. Manually changed code provided by AI to change the constraints on the weights
        # each stock has a min weight of (1/2n)% and a max weight of 15%
        bounds = [(1 / (2 * len(selected_tickers)), 0.15)] * len(selected_tickers)
        
        result = minimize(objective, initial_weights, constraints=constraints, bounds=bounds)
        # print(result.fun)

        
        if result.success:
            weights = result.x
             # Calculate the total cost and fractional shares for each stock
            total_costs = [
                (INVESTMENT * weight - min(3.95, 0.001 * (INVESTMENT * weight / ticker_prices[ticker])))
                for weight, ticker in zip(weights, selected_tickers)
            ]
            shares_purchased = [
                total_cost / ticker_prices[ticker]
                for total_cost, ticker in zip(total_costs, selected_tickers)
            ]
            if result.fun < min_result:
                min_result = result.fun
        
            portfolio_data.append({
                'tickers': ', '.join(selected_tickers), #AI
                'weights': result.x,
                'STD Between Stock and Market Index': result.fun,
                'num_stocks': i
            })
        
    return [pd.DataFrame(portfolio_data), min_result]
    
# all_data['Ticker']
portfolio_df = portfolio_generator(all_data['Ticker'])[0]
min_std = portfolio_generator(all_data['Ticker'])[1]

portfolio_df

$UPS: possibly delisted; no price data found  (1d 2024-11-01 -> 2024-11-01)
$TXN: possibly delisted; no price data found  (1d 2024-11-01 -> 2024-11-01)
$USB: possibly delisted; no price data found  (1d 2024-11-01 -> 2024-11-01)
$UNP: possibly delisted; no price data found  (1d 2024-11-01 -> 2024-11-01)
$BK: possibly delisted; no price data found  (1d 2024-11-01 -> 2024-11-01)
$BB.TO: possibly delisted; no price data found  (1d 2024-11-01 -> 2024-11-01)
$AIG: possibly delisted; no price data found  (1d 2024-11-01 -> 2024-11-01)
$CAT: possibly delisted; no price data found  (1d 2024-11-01 -> 2024-11-01)
$AMZN: possibly delisted; no price data found  (1d 2024-11-01 -> 2024-11-01)
$RY.TO: possibly delisted; no price data found  (1d 2024-11-01 -> 2024-11-01)
$TD.TO: possibly delisted; no price data found  (1d 2024-11-01 -> 2024-11-01)
$AXP: possibly delisted; no price data found  (1d 2024-11-01 -> 2024-11-01)
$AAPL: possibly delisted; no price data found  (1d 2024-11-01 -> 2024-11-01)
$ACN:

""


In [37]:
portfolio_data

[]

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.